In [1]:
"""
This script prepares your Python environment by loading essential tools for handling environment variables, 
interacting with a Neo4j graph database, and utilizing a bespoke graph module, which could be sourced 
from either a local graph class or optionally from the trengx package.

import os: 
This line imports the built-in Python module os, which provides a portable way of using 
operating system dependent functionality, like reading or writing to the environment.

from dotenv import load_dotenv: 
This line imports the load_dotenv function from the dotenv module. 
This function reads a file (by default, a file named .env in the same directory as the script) and 
loads its contents as environment variables.

from neo4j import GraphDatabase as graphdb: 
This line imports the GraphDatabase class from the neo4j module, and renames it to graphdb 
in your script for convenience. This class is used to interact with a Neo4j graph database.

import sys and sys.path.append('../trengx'): 
These lines modify the system path that Python uses to look for modules. The sys module provides access 
to some variables used or maintained by the Python interpreter. In this case, it's used to add 
the parent directory's trengx subdirectory to the list of directories that Python will search when you import a module.

import graph: This line imports the graph module, which is expected to be found in the trengx directory 
that was just added to the system path. This module likely contains the Graph class that you use in your script. 
The specific functionality of this module depends on its implementation, but based on context, 
it likely provides tools for creating and manipulating a computational graph.
"""

# %pip install trengx
# from trengx import graph

import os
from neo4j import GraphDatabase as graphdb
import sys
sys.path.append('../trengx')
import graph

In [2]:
# Retrieve secrets (confidential credentials) from environment variables stored in .env file
uri = os.getenv('NEO4J_URI') # Get the value of the uri variable
user = os.getenv('NEO4J_USER') # Get the value of the user variable
password= os.getenv('NEO4J_PASSWORD')  # Get the value of the pw variable

# Create a Graph class instance
g = graph.Graph(graphdb, uri, user, password)


In [3]:

node = g.create_node ('num', {'name':'jun', 'value':9})  
node['id']

47